In [77]:
def imputardf(df, name):

  import pandas as pd
  import numpy as np
  from scipy.stats import gaussian_kde
  from sklearn.model_selection import train_test_split
  from sklearn.preprocessing import MinMaxScaler
  from sklearn.metrics import silhouette_score
  from sklearn.cluster import KMeans

  ## Borro colls irrelevantes
  df = df.dropna()
  df = df.drop("property_currency", axis=1)
  df = df.drop("operation", axis=1)
  df = df.drop("place_l4", axis=1)
  df = df.drop("place_l5", axis=1)
  df = df.drop("place_l6", axis=1)
  df = df.drop("id", axis=1)
  df = df.drop("start_date", axis=1)
  df = df.drop("end_date", axis=1)
  df = df.drop("created_on", axis=1)


  ## Borro filas con nulls relevantes
  df.dropna(subset=["latitud", "longitud"], inplace=True)
  df = df.dropna(subset=["place_l3"])
  df = df.dropna(subset=["property_bedrooms", "property_rooms"])
  df = df.dropna(subset=["property_surface_covered", "property_surface_total"])

  ## Borro filas con datos erroneos
  df = df[df["property_bedrooms"] <= df["property_rooms"]]
  df = df[df["property_surface_covered"] <= df["property_surface_total"]]

  ## Borro valores atipicos
  mediana = df["property_surface_total"].median()
  limite_superior = mediana * 7
  df = df[df["property_surface_total"] <= limite_superior]

  mediana = df["property_bedrooms"].median()
  limite_superior = mediana * 7
  df = df[df["property_bedrooms"] <= limite_superior]


  ## Creo nuevas cols
  df["total_m2_price"] = df["property_price"] / df["property_surface_total"]
  df["covered_m2_price"] = df["property_price"] / df["property_surface_covered"]



  mediana_total_m2_price = df["total_m2_price"].median()

  limite_inferior = mediana_total_m2_price / 5
  limite_superior = mediana_total_m2_price * 5

  df = df[(df["total_m2_price"] >= limite_inferior) & (df["total_m2_price"] <= limite_superior)]

  q1 = df["covered_m2_price"].quantile(0.25)
  q3 = df["covered_m2_price"].quantile(0.75)
  iqr = q3 - q1

  lower_bound = q1 - 1.5 * iqr
  upper_bound = q3 + 1.5 * iqr

  df = df[(df["covered_m2_price"] >= lower_bound) & (df["covered_m2_price"] <= upper_bound)]


  df.to_csv(name)




In [71]:
df_raw = pd.read_csv(path_df)

In [72]:
tipos_propiedades = ["Departamento", "Casa", "PH"]

df_raw = df_raw[(df_raw["property_currency"] == "USD") & (df_raw["operation"] == "Venta") & (df_raw["property_type"].isin(tipos_propiedades)) &
        (df_raw["place_l2"] == "Capital Federal")]


In [74]:
seed = 123

_df, df_test = train_test_split(df_raw, test_size=0.2, random_state=seed)

df = _df

In [76]:
imputardf(df,"df_train.csv")
imputardf(df_test,"df_test.csv")